## Query 9

Relevancia de autores de reseñas por país. Calcular la relevancia acumulada de autores de reseñas agrupada por país. Una reseña útil tendrá mucho más valor en productos famosos (con muchas ventas) que en productos poco conocidos (con pocas ventas). 

La relevancia se calcula en varios niveles:

**1. Relevancia de producto (por cantidad de ventas):**
```
relevancia_producto = log(1 + total_ventas_producto)
```

**2. Relevancia de reseña (votos favorables ponderados por popularidad del producto):**
```
relevancia_resena = helpful_votes * relevancia_producto
```

**3. Relevancia de autor (suma de todas sus reseñas):**
```
relevancia_autor = sum(relevancia_resena)
cantidad_resenas_autor = count(reseñas)
```

**4. Relevancia por país (suma de todos los autores del país):**
```
relevancia_total_pais = sum(relevancia_autor) para todos los autores del país
cantidad_autores_pais = count(autores únicos del país)
cantidad_resenas_pais = count(reseñas del autor)
```

Esto permite identificar qué países tienen los autores de reseñas más útiles, considerando tanto la calidad de sus reseñas (votos favorables) como su participación en productos populares, además de la cantidad de autores y reseñas por país.

Para hacer esto debemos seguir los siguientes pasos:

1. Importar el dataset de order_items (.data/raw/order_items.csv).
2. Filtrar por valores inválidos (product_id, quantity).
3. Mapear para obtener (product_id, quantity).
4. Reducir por product_id para obtener (product_id, total_ventas_producto).
5. Importar el dataset de reviews (.data/raw/reviews.csv).
6. Filtrar por valores inválidos (product_id, helpful_votes, customer_id).
7. Mapear para obtener (product_id, (customer_id, helpful_votes)).
8. Realizar un join entre reviews y total ventas por producto.
9. Mapear a (customer_id, (relevancia_resena, 1)) donde relevancia_resena = helpful_votes * log(1 + total_ventas_producto), agregando un 1 para contar reseñas.
10. Reducir por customer_id para obtener (customer_id, (relevancia_autor, cantidad_resenas_autor)) sumando relevancias y contando reseñas.
11. Importar el dataset de customers (.data/raw/customers.csv).
12. Filtrar por valores inválidos (customer_id, country).
13. Mapear para obtener (customer_id, country).
14. Realizar un join entre relevancia_autor y customers por customer_id.
15. Mapear a (country, (relevancia_autor, 1, cantidad_resenas_autor)) agregando un 1 para contar autores.
16. Reducir por country para obtener (country, (relevancia_total_pais, cantidad_autores, cantidad_resenas_pais)) sumando relevancias, contando autores y sumando reseñas.
17. Ordenar descendente por relevancia_total_pais y mostrar resultados.

In [ ]:
# Spark session setup
from pyspark.sql import SparkSession
import math

spark = SparkSession.builder.appName("MejoresEscritoresReseñas").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [2]:
items_df = spark.read.csv(
    "../../.data/raw/order_items.csv", header=True, inferSchema=True
)
items_rdd = items_df.rdd

sales_by_product = (
    items_rdd.filter(lambda r: r.product_id and r.quantity is not None)
    .map(lambda r: (r.product_id, r.quantity))  # (product_id: int, quantity: int)
    .reduceByKey(lambda a, b: a + b)  # (product_id, total_ventas_producto)
)

In [3]:
reviews_df = spark.read.csv("../../.data/raw/reviews.csv", header=True, inferSchema=True)
reviews_rdd = reviews_df.rdd

reviews_clean = reviews_rdd.filter(
    lambda r: r.product_id and r.helpful_votes is not None and r.customer_id
).map(
    lambda r: (int(r.product_id), (int(r.customer_id), int(r.helpful_votes)))
)  # (product_id: int, (customer_id: int, helpful_votes: int))

reviews_with_sales = reviews_clean.join(
    sales_by_product
)  # (product_id, ((customer_id, helpful_votes), total_ventas))

In [4]:
relevance_by_author = reviews_with_sales.map(
    lambda kv: (
        kv[1][0][0],  # customer_id
        kv[1][0][1]
        * math.log(
            1 + kv[1][1]
        ),  # relevancia_resena = helpful_votes * log(1 + total_ventas)
    )
).reduceByKey(  # (customer_id: int, relevancia_resena: float)
    lambda a, b: a + b
)  # (customer_id, relevancia_autor)

In [5]:
customers_df = spark.read.csv(
    "../../.data/raw/customers.csv", header=True, inferSchema=True
)
customers_rdd = customers_df.rdd

customers_clean = customers_rdd.filter(lambda r: r.customer_id and r.country).map(
    lambda r: (r.customer_id, r.country.strip().upper())
)  # (customer_id: int, country: str)

authors_with_country = relevance_by_author.join(
    customers_clean
)  # (customer_id, (relevancia_autor, country))

In [6]:
country_stats = authors_with_country.map(
    lambda kv: (kv[1][1], (kv[1][0], 1))
).reduceByKey(  # (country, (relevancia_autor, 1))
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)  # (country, (relevancia_total_pais, cantidad_autores))

reviews_by_country = (
    reviews_with_sales.map(lambda kv: (kv[1][0][0], 1))  # (customer_id, 1)
    .join(customers_clean)
    .map(lambda kv: (kv[1][1], kv[1][0]))  # (customer_id, (1, country))  # (country, 1)
    .reduceByKey(lambda a, b: a + b)
)  # (country, cantidad_resenas)

results = (
    country_stats.join(
        reviews_by_country
    )  # (country, ((relevancia_total, cantidad_autores), cantidad_resenas))
    .map(
        lambda kv: {
            "pais": kv[0],
            "relevancia_total": round(kv[1][0][0], 2),
            "cantidad_autores": kv[1][0][1],
            "cantidad_resenas": kv[1][1],
        }
    )
    .sortBy(lambda x: x["relevancia_total"], ascending=False)
    .collect()
)

In [7]:
# Present results
import pandas as pd

if results:
    df = pd.DataFrame(results)
    display(df)
else:
    print("No results")

,pais,relevancia_total,cantidad_autores,cantidad_resenas
0,FRANCE,168657.51,1198,1282
1,JAPAN,168540.78,1253,1348
2,CANADA,167212.56,1237,1324
3,AUSTRALIA,167094.58,1222,1290
4,BRAZIL,162562.81,1197,1280
5,MEXICO,160838.77,1177,1248
6,GERMANY,157849.14,1187,1269
7,UK,157830.77,1125,1208
8,USA,157743.05,1158,1230
9,INDIA,147406.40,1098,1166
